# AMPLPY: Jupyter Notebook Integration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplpy/blob/master/notebooks/magics.ipynb)

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy

Jupyter Notebooks: https://github.com/ampl/amplpy/tree/master/notebooks

### Setup

In [1]:
!pip install -q amplpy

### Google Colab interagration

In [2]:
%%bash
test ! -z $COLAB_GPU || exit 0  # Run only if running on Google Colab
test ! -f ampl.installed || exit 0  # Run only once
rm -rf ampl.linux-intel64
# You can install a demo bundle with all solvers:
#   curl -O https://portal.ampl.com/dl/amplce/ampl.linux64.tgz && tar xzf ampl.linux64.tgz
# Or pick individual modules (recommended in order to reduce disk usage):
curl -O https://portal.ampl.com/dl/modules/ampl-module.linux64.tgz && tar xzf ampl-module.linux64.tgz
curl -O https://portal.ampl.com/dl/modules/gurobi-module.linux64.tgz && tar xzf gurobi-module.linux64.tgz
cp ampl.linux-intel64/ampl.lic ampl.linux-intel64/ampl.lic.demo
touch ampl.installed

In [3]:
%%bash
# If you have an AMPL Cloud License or an AMPL CE license, you can use it on Google Colab
# Note: Your license UUID should never be shared. Please make sure you delete the license UUID
# and rerun this cell before sharing the notebook with anyone
LICENSE_UUID=
test ! -z $COLAB_GPU || exit 0  # Run only if running on Google Colab
cd ampl.linux-intel64 && pwd
test -z $LICENSE_UUID && cp ampl.lic.demo ampl.lic  # Restore demo license in case LICENSE_UUID is empty
test ! -z $LICENSE_UUID && curl -O https://portal.ampl.com/download/license/$LICENSE_UUID/ampl.lic
./ampl -vvq

In [4]:
import os
if 'COLAB_GPU' in os.environ:
    os.environ['PATH'] += os.pathsep + '/content/ampl.linux-intel64'

### Imports

In [5]:
from amplpy import AMPL, register_magics


### Instantiate an ampl object and register jupyter magics

In [6]:
ampl = AMPL()
# Store %%ampl cells in the list _ampl_cells
# Evaluate %%ampl_eval cells with ampl.eval()
register_magics(store_name='_ampl_cells', ampl_object=ampl)

### Use `%%ampl_eval` to pass the model to AMPL

In [7]:
%%ampl_eval
set SIZES;
param capacity >= 0;
param value {SIZES};
var Qty {SIZES} binary;
maximize TotVal: sum {s in SIZES} value[s] * Qty[s];
subject to Cap: sum {s in SIZES} s * Qty[s] <= capacity;

### Set data

In [8]:
ampl.set['SIZES'] = [5, 4, 6, 3]
ampl.param['value'] = [10, 40, 30, 50]
ampl.param['capacity'] = 10

### Use `%%ampl_eval` to display values

In [9]:
%%ampl_eval
display SIZES;
display value;
display capacity;

set SIZES := 5 4 6 3;

value [*] :=
3  50
4  40
5  10
6  30
;

capacity = 10



### Use amplpy to retrive values

In [10]:
print('SIZES:', ampl.set['SIZES'].getValues().toList())
print('value:', ampl.param['value'].getValues().toDict())
print('capacity:', ampl.param['capacity'].value())

SIZES: [5.0, 4.0, 6.0, 3.0]
value: {3.0: 50.0, 4.0: 40.0, 5.0: 10.0, 6.0: 30.0}
capacity: 10.0


### Use `%%ampl_eval` to solve the model

In [11]:
%%ampl_eval
option solver gurobi;
option gurobi_options 'outlev=1';
solve;

Gurobi 9.5.0: outlev=1
Set parameter OutputFlag to value 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x1dc33b55
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [3e+00, 6e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 50.0000000
Presolve removed 1 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 90 50 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Set parameter Presolve to value 0
Set parameter Method t